In [1]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.optim import lr_scheduler

if torch.cuda.is_available():
    # Set Device
    torch.cuda.set_device(0)
    print("Cuda enabled on device: {}".format(torch.cuda.current_device()))



Cuda enabled on device: 0


In [18]:
#%%writefile depthnet.py
# %load depthnet.py
import torch
import torch.nn as nn
from torch.nn import init

class DepthNet(nn.Module):
    def __init__(self, input_nc, output_nc, norm_layer=nn.BatchNorm2d):
        super(DepthNet, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        use_bias = True

        # Conv1
        model1=[nn.Conv2d(input_nc, 64, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]
        # add a subsampling operation (in self.forward())

        # Conv2
        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]
        # add a subsampling layer operation (in self.forward())

        # Conv3
        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]
        # add a subsampling layer operation

        # Conv4
        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        # Conv5
        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        # Conv6
        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        # Conv7
        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        # Conv7
        model8up=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=use_bias)]
        model3short8=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]

        model8=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model8+=[nn.ReLU(True),]
        model8+=[norm_layer(256),]

        # Conv9
        model9up=[nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=use_bias),]
        model2short9=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        # add the two feature maps above        

        model9=[nn.ReLU(True),]
        model9+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model9+=[nn.ReLU(True),]
        model9+=[norm_layer(128),]

        # Conv10
        model10up=[nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1, bias=use_bias),]
        model1short10=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        # add the two feature maps above

        model10=[nn.ReLU(True),]
        model10+=[nn.Conv2d(128, 128, kernel_size=3, dilation=1, stride=1, padding=1, bias=use_bias),]
        model10+=[nn.LeakyReLU(negative_slope=.2),]

        # Depth Map Regression Output
        model_out=[nn.Conv2d(128, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=use_bias),]
        model_out+=[nn.ReLU(True),] # Depth should be in [0, +inf)

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8up = nn.Sequential(*model8up)
        self.model8 = nn.Sequential(*model8)
        self.model9up = nn.Sequential(*model9up)
        self.model9 = nn.Sequential(*model9)
        self.model10up = nn.Sequential(*model10up)
        self.model10 = nn.Sequential(*model10)
        self.model3short8 = nn.Sequential(*model3short8)
        self.model2short9 = nn.Sequential(*model2short9)
        self.model1short10 = nn.Sequential(*model1short10)
        self.model_out = nn.Sequential(*model_out)

    def forward(self, input_A):
#         conv1_2 = self.model1(torch.cat((input_A,input_B,mask_B),dim=1))
        conv1_2 = self.model1(input_A)
        conv2_2 = self.model2(conv1_2[:,:,::2,::2]) # downsample
        conv3_3 = self.model3(conv2_2[:,:,::2,::2]) # downsample
        conv4_3 = self.model4(conv3_3[:,:,::2,::2]) # downsample
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)
        conv8_up = self.model8up(conv7_3) + self.model3short8(conv3_3) # Shortcut
        conv8_3 = self.model8(conv8_up)
        conv9_up = self.model9up(conv8_3) + self.model2short9(conv2_2) # Shortcut
        conv9_3 = self.model9(conv9_up)
        conv10_up = self.model10up(conv9_3) + self.model1short10(conv1_2) # Shortcut
        conv10_2 = self.model10(conv10_up)
        out_reg = self.model_out(conv10_2)

        return out_reg


In [25]:
class DepthNetWithHints(nn.Module):
    def __init__(self, depthnet, hist_len, bin_size, num_hints_layers, norm_layer=nn.BatchNorm2d):
        """Takes an existing DepthNet, along with the size of the histogram and the size of its bins"""
        super(DepthNetWithHints, self).__init__()
        self.input_nc = depthnet.input_nc
        self.output_nc = depthnet.output_nc
        self.depthnet = depthnet
        self.hist_len = hist_len
        self.num_hints_layers = num_hints_layers
        self.norm_layer = norm_layer
        
        # Create hints network
        assert num_hints_layers > 0
        # Extract number of out channels of conv4
        hints_output = depthnet.model4[0].out_channels
        hints = [nn.Conv2d(hist_len, hints_output, kernel_size=1),]
        hints += [nn.ReLU(True),]
        for _ in range(num_hints_layers-1):
            hints = [nn.Conv2d(hints_output, hints_output, kernel_size=1),]
            hints += [nn.ReLU(True),]
        
        self.global_hints = nn.Sequential(*hints)
        
    def forward(self, input_A, hist):
        # |hist| should be a (1, hist_len, 1, 1) tensor
        # First 4 layers of regular depthnet
        conv1_2 = self.depthnet.model1(input_A)
        conv2_2 = self.depthnet.model2(conv1_2[:,:,::2,::2]) # downsample
        conv3_3 = self.depthnet.model3(conv2_2[:,:,::2,::2]) # downsample
        conv4_3 = self.depthnet.model4(conv3_3[:,:,::2,::2]) # downsample
        # Global hints network
        hints_out = self.global_hints(hist)
        # Replicate and add to output of conv4 (broadcasting takes care of this)
        conv5_3 = self.depthnet.model5(conv4_3 + hints_out)
        
        # Finish doing the rest of the depthnet
        conv6_3 = self.depthnet.model6(conv5_3)
        conv7_3 = self.depthnet.model7(conv6_3)
        conv8_up = self.depthnet.model8up(conv7_3) + self.model3short8(conv3_3) # Shortcut
        conv8_3 = self.depthnet.model8(conv8_up)
        conv9_up = self.depthnet.model9up(conv8_3) + self.model2short9(conv2_2) # Shortcut
        conv9_3 = self.depthnet.model9(conv9_up)
        conv10_up = self.depthnet.model10up(conv9_3) + self.model1short10(conv1_2) # Shortcut
        conv10_2 = self.depthnet.model10(conv10_up)
        out_reg = self.depthnet.model_out(conv10_2)
        return out_reg
         
        
        
        

In [19]:
dn = DepthNet(3, 1)

In [27]:
dnwh = DepthNetWithHints(dn, 10, 10, 4)

In [28]:
for p in dnwh.parameters():
    print(p)

Parameter containing:
tensor([[[[ 0.1670, -0.1430,  0.0610],
          [-0.0331,  0.1254, -0.1869],
          [ 0.1453, -0.1541, -0.0664]],

         [[-0.1253, -0.1211,  0.0437],
          [ 0.1084,  0.1623,  0.0205],
          [ 0.1579,  0.1094, -0.1339]],

         [[ 0.0753, -0.0813, -0.0735],
          [ 0.0817,  0.0916,  0.0280],
          [-0.0562,  0.1419,  0.0741]]],


        [[[ 0.0932, -0.0577,  0.0946],
          [-0.1140,  0.0536, -0.1719],
          [ 0.1014, -0.0458, -0.1626]],

         [[ 0.1616, -0.0554,  0.1531],
          [-0.1059,  0.0739,  0.0871],
          [-0.0556, -0.1221,  0.1287]],

         [[ 0.1572, -0.1391, -0.1112],
          [-0.0815,  0.1790,  0.0005],
          [-0.0503, -0.0236,  0.0217]]],


        [[[-0.0182, -0.1371, -0.1787],
          [-0.0166,  0.1288, -0.1562],
          [ 0.0247,  0.1817, -0.1902]],

         [[-0.1005,  0.0516, -0.0095],
          [-0.1897,  0.0205, -0.0196],
          [ 0.1773,  0.0893,  0.0905]],

         [[-0.1226,  0

In [40]:
a = torch.zeros(4, 2, 3, 3)
b = torch.Tensor([[[[5]], [[4]]]])

In [43]:
c = a+b
c[0, 1, :, :]

tensor([[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]])